In [1]:
# import libraries
import pandas as pd 
import datetime as dt
import time
from datetime import datetime
import seaborn as sns

## Bitcoin data

In [2]:
df1 = pd.read_csv('BitCoinData.csv', parse_dates=['Date'])
df1.head()

,Date,price
0,2021-02-10,44918.183594
1,2021-02-11,47909.332031
2,2021-02-12,47504.851562
3,2021-02-13,47105.515625
4,2021-02-14,48717.289062


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    214 non-null    datetime64[ns]
 1   price   214 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.5 KB


In [4]:
df_btc = df1.copy()

In [5]:
df_btc= df_btc.set_index('Date')

In [7]:
df_btc.head()

,price
Date,
2021-02-10,44918.183594
2021-02-11,47909.332031
2021-02-12,47504.851562
2021-02-13,47105.515625
2021-02-14,48717.289062


## Twitter

In [ ]:
df2['date_dt']=pd.to_datetime(df2.date, errors='coerce')
df2['date_dt'].isna().sum()

In [ ]:
df2=df2.dropna(subset=['date_dt'], axis=0)

In [ ]:
df2['date_dt'].isna().sum()

In [ ]:
pd.to_pickle(df2, 'clean_tweets.pkl')

In [ ]:
df_pickle = pd.read_pickle('clean_tweets.pkl')
df_pickle.info()

In [ ]:
df_tweets= df_pickle.copy()

In [ ]:
# drop the time stamp
df_tweets['date_dt']= pd.to_datetime(df_tweets['date_dt'].dt.date)

In [ ]:
df

In [ ]:
df_tweets=df_tweets.set_index('date_dt')

In [ ]:
df_tweets

In [ ]:
df.info()

In [ ]:
# Setting the timestamp as the index

In [ ]:
# Just perform a join and that's it

## Merge

In [ ]:
df_merged = pd.merge(df_tweets, df_btc, how="left", left_index=True, right_index=True)

In [ ]:
df_merged.price.isna().sum()

In [ ]:
df_merged=df_merged.dropna(subset=['price'])

In [ ]:
df_merged

In [ ]:
df_merged=df_merged.reset_index().rename(columns={'index': 'date_only'})

In [ ]:
df_merged.head()

In [ ]:
# normalizing only the price
def mn_scale(series):
    return (series - series.min()) / (series.max() - series.min())



In [ ]:
df_merged['price_norm'] =mn_scale(df_merged.price)

In [ ]:
(df_merged
 .groupby('date_only')
 .agg({'subjectivity': 'mean', 'polarity': 'mean', 'price_norm': 'mean'})
 #.reset_index()
 .plot()
)

## Transform the data

In [ ]:
df.info()

In [ ]:
# Identify all the categorical columns 
[col for col in df.columns if not col in df._get_numeric_data().columns]

In [ ]:
# save a new subset -> numerical columns as the new DF
numerical = df._get_numeric_data()
numerical

In [ ]:
# save a new subset -> cathegorical columns as the new DF
categorical=[i for i in df.columns if df.dtypes[i]=='object']
categorical

In [ ]:
# split between the independent variables and the target variable
X=numerical.drop('total_claim_amount', axis=1)
y=numerical.total_claim_amount

In [ ]:
# scaling the data and splitting it into train and test
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns = X.columns)
X_test  = pd.DataFrame(X_test , columns = X.columns)

In [ ]:
## Build a pipeline for validation metrics

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
# preparing data for linear regression

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

ols = LinearRegression(normalize=True)
ols.fit(X_train,y_train)

# predictions for our model
pred = ols.predict(X_test)

# regression evaluation metrics
test_pred = ols.predict(X_test)
train_pred = ols.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

## Model Validation

In [ ]:
# print the intercept
print(ols.intercept_)

In [ ]:
coeff_df = pd.DataFrame(ols.coef_, X.columns, columns=['Coefficient'])
coeff_df

## Polynomial Regression

In [ ]:
# One common pattern within machine learning is to use linear models
#trained on nonlinear functions of the data. This approach maintains
#the generally fast performance of linear methods, while allowing them
# to fit a much wider range of data.

from sklearn.preprocessing import PolynomialFeatures

poly_reg = PolynomialFeatures(degree=2)

X_train_2_d = poly_reg.fit_transform(X_train)
X_test_2_d = poly_reg.transform(X_test)

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(X_train_2_d,y_train)

test_pred = lin_reg.predict(X_test_2_d)
train_pred = lin_reg.predict(X_train_2_d)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Polynomail Regression", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

## Stochastic Gradient Descent

In [ ]:
 '''Gradient Descent is a very generic optimization algorithm capable of
 finding optimal solutions to a wide range of problems. The general
 idea of Gradient Sescent is to tweak parameters iteratively in order
 to minimize a cost function. Gradient Descent measures the local gradient
 of the error function with regards to the parameters vector, and it goes 
 in the direction of descending gradient. Once the gradient is zero, you
 have reached a minimum.'''

from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(n_iter_no_change=250, penalty=None, eta0=0.0001, max_iter=100000)
sgd_reg.fit(X_train, y_train)

test_pred = sgd_reg.predict(X_test)
train_pred = sgd_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Stochastic Gradient Descent", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

## Artficial Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential()

model.add(Dense(X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(optimizer=Adam(0.00001), loss='mse')

r = model.fit(X_train, y_train,
              validation_data=(X_test,y_test),
              batch_size=1,
              epochs=100)

In [ ]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Artficial Neural Network", *evaluate(y_test, test_pred), 0]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', 'Cross Validation'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

## Models Comparison

In [ ]:
results_df.set_index('Model', inplace=True)
results_df['R2 Square'].plot(kind='barh', figsize=(12, 8))